In [1]:
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from bofire.data_models.features.api import CategoricalInput
from bofire.data_models.features.api import ContinuousOutput
from bofire.data_models.objectives.api import MaximizeObjective
from bofire.data_models.domain.api import Inputs, Outputs
from bofire.data_models.domain.api import Domain
from bofire.data_models.strategies.api import QnehviStrategy
from bofire.strategies.mapper import map
from bofire.data_models.surrogates.api import  SingleTaskGPSurrogate, RandomForestSurrogate
from bofire.data_models.surrogates.api import  BotorchSurrogates
from bofire.data_models.surrogates.scaler import ScalerEnum


/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x1_options = ['a', 'b', 'c']
x2_options = ['d', 'e', 'f']
x3_options = ['g', 'h', 'i']

# initialize an empty dictionary to store experiment data
experiments_data = {
    'x1': [],
    'x2': [],
    'x3': [],
    'f_1': [],
    'f_2': [],
    'f_3': [],
    'valid_f_1': [],
    'valid_f_2': [],
    'valid_f_3': []
}

# generate 30 experiments
for _ in range(30):
    experiments_data['x1'].append(random.choice(x1_options))
    experiments_data['x2'].append(random.choice(x2_options))
    experiments_data['x3'].append(random.choice(x3_options))
    experiments_data['f_1'].append(round(random.uniform(0.0, 1.0), 2))
    experiments_data['f_2'].append(round(random.uniform(0.0, 1.0), 2))
    experiments_data['f_3'].append(round(random.uniform(0.0, 1.0), 2))
    experiments_data['valid_f_1'].append(1)
    experiments_data['valid_f_2'].append(1)
    experiments_data['valid_f_3'].append(1)

experiments = pd.DataFrame.from_dict(experiments_data)
experiments

,x1,x2,x3,f_1,f_2,f_3,valid_f_1,valid_f_2,valid_f_3
0,c,f,g,0.15,0.24,0.69,1,1,1
1,c,e,g,0.61,0.35,0.24,1,1,1
2,a,d,i,0.23,0.37,0.29,1,1,1
3,b,e,h,0.59,0.81,0.14,1,1,1
4,b,e,h,0.93,0.81,0.69,1,1,1
5,c,d,g,0.45,0.00,0.50,1,1,1
6,b,e,h,0.65,0.39,0.24,1,1,1
7,b,d,i,0.13,0.72,0.05,1,1,1
8,a,d,i,0.02,0.08,0.85,1,1,1
9,c,e,i,0.25,0.56,0.45,1,1,1


In [3]:
# setting up the input space
x1 = CategoricalInput(key = "x1", categories = ['a', 'b', 'c'])
x2 = CategoricalInput(key = "x2", categories = ['d', 'e', 'f'])
x3 = CategoricalInput(key = "x3", categories = ['g', 'h', 'i'])

# setting up the output space
objective = MaximizeObjective(
    w=1.0, 
    bounds= [0.0,1.0],
)
f_1 = ContinuousOutput(key="f_1", objective=objective)
f_2 = ContinuousOutput(key="f_2", objective=objective)
f_3 = ContinuousOutput(key="f_3", objective=objective)

# putting everything together
input_features = Inputs(features = [x1, x2, x3])
output_features = Outputs(features = [f_1, f_2, f_3])

domain = Domain(
    inputs=input_features, 
    outputs=output_features
    )

domain.get_feature_reps_df()

,Type,Description
x1,CategoricalInput,3 categories
x2,CategoricalInput,3 categories
x3,CategoricalInput,3 categories
f_1,ContinuousOutput,ContinuousOutputFeature
f_2,ContinuousOutput,ContinuousOutputFeature
f_3,ContinuousOutput,ContinuousOutputFeature


In [12]:
mobo_strategy_data_model = QnehviStrategy(domain=domain, 
    surrogate_specs=BotorchSurrogates(
        surrogates=[SingleTaskGPSurrogate(inputs=domain.inputs, outputs=domain.outputs.get_by_keys([key]), scaler=ScalerEnum.IDENTITY) for key in ["f_1", "f_2", "f_3"]]))
mobo_strategy = map(mobo_strategy_data_model)
mobo_strategy.tell(experiments = experiments)
mobo_strategy.ask(1)

,x1,x2,x3,f_1_pred,f_2_pred,f_3_pred,f_1_sd,f_2_sd,f_3_sd,f_1_des,f_2_des,f_3_des
0,b,e,i,0.413319,0.428675,0.440691,0.296142,0.320579,0.288853,0.413319,0.428675,0.440691


In [4]:
# Note that you can also modify other hyperparameters of the used RF, just have a look at RandomForestSurrogate data model

mobo_strategy_data_model = QnehviStrategy(domain=domain, seed=42,
    surrogate_specs=BotorchSurrogates(
        surrogates=[RandomForestSurrogate(inputs=domain.inputs, outputs=domain.outputs.get_by_keys([key]), n_estimators=10, random_state=42) for key in ["f_1", "f_2", "f_3"]]))
mobo_strategy = map(mobo_strategy_data_model)
mobo_strategy.tell(experiments = experiments)
mobo_strategy.ask(1)

,x1,x2,x3,f_1_pred,f_2_pred,f_3_pred,f_1_sd,f_2_sd,f_3_sd,f_1_des,f_2_des,f_3_des
0,b,e,g,0.6545,0.768,0.483033,0.099033,0.165784,0.220996,0.6545,0.768,0.483033


In [6]:
mobo_strategy_data_model = QnehviStrategy(domain=domain, seed=42,
    surrogate_specs=BotorchSurrogates(
        surrogates=[RandomForestSurrogate(inputs=domain.inputs, outputs=domain.outputs.get_by_keys([key]), n_estimators=10, random_state=42) for key in ["f_1", "f_2", "f_3"]]))
mobo_strategy = map(mobo_strategy_data_model)
mobo_strategy.tell(experiments = experiments)
mobo_strategy.ask(1)

,x1,x2,x3,f_1_pred,f_2_pred,f_3_pred,f_1_sd,f_2_sd,f_3_sd,f_1_des,f_2_des,f_3_des
0,b,e,g,0.6545,0.768,0.483033,0.099033,0.165784,0.220996,0.6545,0.768,0.483033
